In [1]:
import sys
import random
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from wavenet import *
from sklearn.model_selection import StratifiedKFold, KFold
from tqdm.notebook import tqdm
from scipy import io
from mpl_toolkits.axes_grid1 import make_axes_locatable

%load_ext autoreload
%matplotlib inline
%precision 3
sys.path.append('../')

# device = "cuda" if torch.cuda.is_available() else "cpu"
device = "cpu"

print(device)
BATCH_SIZE = 8
EPOCHS = 100
SEED = 42
lr = 1e-3
# if you need to reload to apply changeing function, you can use this 
%autoreload

# from original_variables import B4_Z, corr_data4_idx

cpu


In [2]:
B_Z = pd.read_csv('B4_Z.csv', index_col=0).T
B_Z.describe()

,0,1,2,3,4,5,6,7,8,9,...,40,41,42,43,44,45,46,47,48,49
count,2.669400e+04,2.669400e+04,2.669400e+04,2.669400e+04,2.669400e+04,2.669400e+04,2.669400e+04,2.669400e+04,2.669400e+04,2.669400e+04,...,2.669400e+04,2.669400e+04,2.669400e+04,2.669400e+04,2.669400e+04,2.669400e+04,2.669400e+04,2.669400e+04,2.669400e+04,2.669400e+04
mean,1.235078e-15,6.941992e-16,2.555335e-17,-2.257212e-15,-5.706914e-16,1.358586e-15,4.331292e-15,-3.162227e-16,3.321935e-16,1.309609e-15,...,2.704396e-16,2.832163e-15,9.284382e-16,-2.326419e-15,-9.635741e-17,-3.918180e-16,1.128606e-15,1.925019e-15,2.235918e-17,1.618379e-16
std,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,...,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00
min,-2.058446e+00,-2.092422e+00,-1.997026e+00,-1.668652e+00,-1.801076e+00,-1.417576e+00,-1.677417e+00,-1.834140e+00,-1.924426e+00,-1.725664e+00,...,-2.485807e+00,-3.058671e+00,-2.053440e+00,-1.930758e+00,-2.249340e+00,-1.503422e+00,-2.394259e+00,-2.067178e+00,-2.644919e+00,-2.470368e+00
25%,-4.699158e-01,-5.510449e-01,-6.145777e-01,-5.662558e-01,-5.554222e-01,-6.442780e-01,-7.347827e-01,-4.576305e-01,-5.569729e-01,-6.326598e-01,...,-6.266575e-01,-6.611715e-01,-5.483359e-01,-6.047582e-01,-6.229022e-01,-6.365820e-01,-6.211954e-01,-5.521250e-01,-6.392076e-01,-5.822687e-01
50%,-1.825097e-01,-2.159564e-01,-2.651920e-01,-3.112324e-01,-2.892532e-01,-4.282878e-01,-4.167227e-01,-1.732174e-01,-2.140049e-01,-3.488928e-01,...,-1.866651e-01,-8.023465e-02,-2.786751e-01,-2.992419e-01,-2.030998e-01,-3.724305e-01,-1.637034e-01,-2.309297e-01,-1.913624e-01,-1.616123e-01
75%,1.685148e-01,1.976680e-01,2.891797e-01,1.001024e-01,1.119641e-01,2.672621e-01,5.303100e-01,1.593763e-01,2.261374e-01,2.402691e-01,...,3.724913e-01,5.500360e-01,1.023709e-01,1.977409e-01,3.586932e-01,2.732392e-01,3.968476e-01,1.903251e-01,3.755492e-01,3.571590e-01
max,7.763553e+00,6.010963e+00,7.205731e+00,5.621428e+00,6.675084e+00,5.170766e+00,4.920170e+00,9.680590e+00,7.891894e+00,5.362618e+00,...,6.646375e+00,7.876607e+00,7.057272e+00,6.019823e+00,6.873250e+00,5.308989e+00,9.428258e+00,6.686560e+00,6.168876e+00,1.265253e+01


In [3]:
look_back = 100
time_conv_frame = 64
fold = 0

X_idx = np.arange(49)
n_feature = len(X_idx)
X = B_Z.iloc[:,X_idx]
X_train, X_valid, X_test = train_val_test(X, look_back, fold)
print(X_train.shape, X_valid.shape, X_test.shape)

y_idx = np.array([49])
y = pd.DataFrame(B_Z.iloc[:,y_idx])
y_train, y_valid, y_test = train_val_test(y, look_back, fold)
print(y_train.shape, y_valid.shape, y_test.shape)

train_dataset = conv2dDataset(X_train, y_train)
valid_dataset = conv2dDataset(X_valid, y_valid)
test_dataset = conv2dDataset(X_test, y_test)

train_dataloader = DataLoader(train_dataset, BATCH_SIZE, shuffle=True)
valid_dataloader = DataLoader(valid_dataset, BATCH_SIZE, shuffle=False)
test_dataloader = DataLoader(test_dataset, BATCH_SIZE, shuffle=False)

(133, 49, 100) (66, 49, 100) (67, 49, 100)
(133, 100) (66, 100) (67, 100)


In [4]:
in_channels = 1
out_channels = 32
dilation = 1

if time_conv_frame % 2 == 0:
    time_conv_frame += 1
padding = (0, (time_conv_frame - 1) // 2)
kernel_size = (n_feature, time_conv_frame)
print(f"kernel size:{time_conv_frame}, time_conv_frame:{time_conv_frame}, padding:{padding}")
conv2d = nn.Conv2d(in_channels, out_channels, kernel_size, padding=padding).to(device)

_kernel_size = 2
dilation = 1
padding = (_kernel_size - 1) * dilation
print(f"_kernel size:{_kernel_size}, padding:{padding}")
causal = nn.Conv1d(in_channels=out_channels, out_channels=16, kernel_size=_kernel_size, padding=padding).to(device)
for x, y in train_dataloader:
    x = x.to(device)
    y = y.to(device)
    print(f"x:{x.shape}")
    x_hat = conv2d(x)
    print(f"x_hat:{x_hat.shape}")
    x_hat = x_hat.squeeze()
    _x_hat = causal(x_hat)
    break

print(f"_x_hat:{_x_hat.shape}")

kernel size:65, time_conv_frame:65, padding:(0, 32)
_kernel size:2, padding:1
x:torch.Size([8, 1, 49, 100])
x_hat:torch.Size([8, 32, 1, 100])
_x_hat:torch.Size([8, 16, 101])


In [5]:
time_conv_frame = 64
model = WaveNet_conv2d(n_resch=512, n_skipch=256, n_feature=n_feature,
                 dilation_depth=5, dilation_repeat=3, kernel_size=2,
                 in_channels=in_channels, out_channels=out_channels, time_conv_frame=time_conv_frame)
model.to(device)
for x, y in train_dataloader:
    x = x.to(device)
    y = y.to(device)  
    y_hat = model(x)
    break
print(f"x:{x.shape}, y:{y.shape}, y_hat:{y_hat.shape}")

x:torch.Size([8, 1, 49, 100]), y:torch.Size([8, 100]), y_hat:torch.Size([8, 100])


In [6]:
def train_wavenet_conv2d(model, train_dataset, valid_dataset, test_dataset, device="cuda", 
                      BATCH_SIZE=16, EPOCHS=100, lr=1e-3,
                     model_path="ex.pth", log_path="ex.csv"):
    """
    Training WaveNet and predict test data
    calculate local to local per one cell by WaveNet structure.
    In this function, finally, you can get across cluster's correlation matrix in test data. 
    
    detail of rule of calculation
        input : X (Tensor): Doudle tensor variable with the shape (B, in_channels, n_feature, T).
        output : series y (batch, time)
        loss function : MSE.
        metric : correlation which axis is time course. 
    Paramns
    ----------------------------------
    dataloader :dataset 
        {X (n_feature, look_back), y (look_back)}
    model_path : str, default "ex.pth"
        path where you save the model
    log_path : str, default "ex.csv"
        path where you save the log
    
    Returns
    ----------------------------------
    test_corr : float
        correlation of test_true and test_pred
    test_preds : ndarray
        output of WaveNeet from test data
    log_df : DataFrame
        log of this model ["loss", "corr", "val_loss", "val_corr", "lr"]
    """
    print(model_path)
    print(log_path)
    train_dataloader = DataLoader(train_dataset, BATCH_SIZE, shuffle=True)
    valid_dataloader = DataLoader(valid_dataset, BATCH_SIZE, shuffle=False)
    test_dataloader = DataLoader(test_dataset, BATCH_SIZE, shuffle=False)

    
    model = model.to(device)
    early_stopping = EarlyStopping(patience=20, device=device, checkpoint_path=model_path)

    criterion = nn.MSELoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=lr)

    schedular = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='max', patience=5, factor=0.8)
    avg_train_losses, avg_valid_losses = [], []
    cols = ["loss", "corr", "val_loss", "val_corr", "lr"]
    log_df = pd.DataFrame(columns=cols)
    cnt = 0

    for epoch in tqdm(range(EPOCHS)):
        #seed_everything(SEED)
        train_losses, valid_losses = [], []
        tr_loss_cls_item, val_loss_cls_item = [], []
        model.train()  # prep model for training
        train_preds, train_true = torch.Tensor([]).to(device), torch.Tensor([]).to(device)
        cnt = 0
        for x, y in train_dataloader:
            x = x.to(device)
            y = y.to(device)
            b_size = x.size(1)
            if len(b_size) == 2:
                x = x.unsqueeze(0)

            optimizer.zero_grad()
            predictions = model(x)
            #loss = criterion( predictions[:, model.receptive_field:].contiguous().view(-1, 256), y[:, model.receptive
            loss = criterion(predictions.contiguous().view(-1), y.contiguous().view(-1))

            # backward pass: compute gradient of the loss with respect to model parameters
            loss.backward()
            # perform a single optimization step (parameter update
            optimizer.step()
            # record training loss
            train_losses.append(loss.item())
            train_true = torch.cat([train_true, y.contiguous().view(-1)], 0)
            train_preds = torch.cat([train_preds, predictions.contiguous().view(-1)], 0)
            cnt += 1
        model.eval()  # prep model for evaluation
        val_preds, val_true = torch.Tensor([]).to(device), torch.Tensor([]).to(device)
        with torch.no_grad():
            for x, y in valid_dataloader:
                x = x.to(device)#.reshape(-1, n_feature, look_back)
                y = y.to(device)
                predictions = model(x)
                #loss = criterion(predictions[:, model.receptive_field:].contiguous().view(-1, 256), y[:, model.receptive_field:].contiguous().view(-1))
                loss = criterion(predictions.contiguous().view(-1), y.contiguous().view(-1))
                valid_losses.append(loss.item())
                val_true = torch.cat([val_true, y.contiguous().view(-1)], 0)
                val_preds = torch.cat([val_preds, predictions.contiguous().view(-1)], 0)
        # calculate average loss over an epoch
        train_loss = np.average(train_losses)
        valid_loss = np.average(valid_losses)
        train_corr = np.corrcoef(x=train_true.cpu().numpy(), y=train_preds.cpu().detach().numpy())[0, 1]
        valid_corr = np.corrcoef(x=val_true.cpu().numpy(), y=val_preds.cpu().detach().numpy())[0, 1]
        print("Epoch : {}, lr:{:0.7f}, ".format(epoch,optimizer.param_groups[0]['lr']) +\
            "loss: {:0.5f}, corr: {:0.5f}, val_loss: {:0.5f}, val_corr: {:.5f}".format(train_loss, train_corr, valid_loss, valid_corr))
        schedular.step(valid_corr)
        
        #logを保存する
        tmp = pd.DataFrame([[train_loss, train_corr, valid_loss, valid_corr, optimizer.param_groups[0]['lr']]],columns=cols)
        log_df = pd.concat([log_df, tmp], axis=0).reset_index(drop=True)
        log_df.to_csv(log_path, index=False)
        if train_corr >= 0.5:
            res = early_stopping(valid_corr, model, mode="max", epoch=epoch)
            cnt = 1
            if  res == 2:
                print("Early Stopping")
                print('folder %d global best val max corr %f' % (fold, early_stopping.best_score))
                break
        if (epoch == EPOCHS - 1) & (cnt == 0):
            res = early_stopping(valid_corr, model, mode="max", epoch=epoch)

    test_preds, test_true = torch.Tensor([]).to(device), torch.Tensor([]).to(device)

    model.load_state_dict(torch.load(model_path, map_location=device))
    model = model.to(device)

    model.eval()
    pred_list = []
    with torch.no_grad():
        for x, y in test_dataloader:
            x = x.to(device)
            y = y.to(device)
            predictions = model(x)
            test_true = torch.cat([test_true, y.reshape(-1)], 0)
            test_preds = torch.cat([test_preds, predictions.reshape(-1,)], 0)
    test_true = test_true.cpu().numpy()
    test_preds = test_preds.cpu().detach().numpy()
    test_corr = np.corrcoef(x=test_true, y=test_preds)[0, 1]
    print(f" best Epoch:{early_stopping.epoch:}, test_corr:{test_corr:.4f}")
    return test_corr, test_preds, log_df

SyntaxError: invalid syntax (<ipython-input-6-0536f9bdc483>, line 62)

In [ ]:
in_channels = 1
out_channels = 32
n_feature = 49
time_conv_frame = 64
model = WaveNet_conv2d(n_resch=512, n_skipch=256, n_feature=n_feature,
                 dilation_depth=5, dilation_repeat=3, kernel_size=2,
                 in_channels=in_channels, out_channels=out_channels, time_conv_frame=time_conv_frame)
model.to(device)
test_corr, test_preds, log_df = train_wavenet_conv2d(model, train_dataset, valid_dataset, test_dataset, device="cpu", 
                      BATCH_SIZE=16, EPOCHS=100, lr=1e-3,
                     model_path="ex.pth", log_path="ex.csv")

In [ ]:
test_corr

In [ ]:
log_df.plot()

In [ ]:
plt.plot(test_preds)

In [21]:
a = np.arange(20).reshape(10, 2)
print(a)
print(a.argmax(axis=1))

[[ 0  1]
 [ 2  3]
 [ 4  5]
 [ 6  7]
 [ 8  9]
 [10 11]
 [12 13]
 [14 15]
 [16 17]
 [18 19]]
[1 1 1 1 1 1 1 1 1 1]


In [12]:
a = (1,2,3)
len(a)

3